# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import numpy as np
import pandas as pd
import pickle
import re
import nltk
import time
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report, accuracy_score, recall_score, precision_score, f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
import datetime
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import make_scorer, accuracy_score, f1_score, fbeta_score, classification_report
from scipy.stats.mstats import gmean
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Essam\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Essam\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Essam\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [2]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('df', engine)
X = df['message']
Y = df.iloc[:,4:]

In [3]:
#Check data
X.sample(5)

1318     after how muche time does the earthquake stop?...
11896    That was three in the last half hour a 7.2 and...
14178    Lutheran World Relief is channeling funds thro...
17896    Vietnam's Red River Delta region is vulnerable...
4610     We don't have water, food, and shelter. we fou...
Name: message, dtype: object

In [4]:
#Check data
Y.head()

,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,water,...,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report,direct,news
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,0,0,1,0,0,0,0,0,0,0,...,1,0,1,0,0,0,0,0,1,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,1,0,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


### 2. Write a tokenization function to process your text data

In [5]:
def tokenize(text):
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex, text)
    
    # Convert text to lowercase and remove punctuation
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")
    # Tokenize words
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

In [6]:
#test
print("text:",X[1],"\ntokens:", tokenize(X[1]))

('text:', u'Is the Hurricane over or is it not over', '\ntokens:', [u'is', u'the', u'hurricane', u'over', u'or', u'is', u'it', u'not', u'over'])


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [7]:
#create pipeline
pipeline = Pipeline([
    ('vectorizer', CountVectorizer(tokenizer=tokenize)), #create the CountVectorizer object
    ('tfidf', TfidfTransformer()), #create Tfidftransformer object    
    ('clf', MultiOutputClassifier(RandomForestClassifier(), n_jobs=-1)) #create RandomForestClassifier
]) 

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [8]:
#to calculate the time of running the train
start_time  = datetime.datetime.now()
#perform train / test split
X_train, X_test, y_train, y_test = train_test_split(X.as_matrix(), Y.as_matrix(), test_size=0.2, random_state=42)
#Train pipeline
pipeline.fit(X_train, y_train)
end_time = datetime.datetime.now()
print ('Training of Pipeline Done..., Time: %d' % ((end_time - start_time).seconds/60),'minutes')

C:\Users\Essam\Anaconda2\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until


('Training of Pipeline Done..., Time: 1', 'minutes')


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [9]:
# test the classifier
y_pred = pipeline.predict(X_test)

In [10]:
#Convert prediction numpy into dataframe
y_test=pd.DataFrame(data=y_test,columns=Y.columns)
y_pred=pd.DataFrame(data=y_pred,columns=Y.columns)


#### DEFINE GRIDSEARCH SCORING TARGET

In [11]:
def multioutput_fscore(y_true,y_pred,beta=1):
    score_list = []
    if isinstance(y_pred, pd.DataFrame) == True:
        y_pred = y_pred.values
    if isinstance(y_true, pd.DataFrame) == True:
        y_true = y_true.values
    for column in range(0,y_true.shape[1]):
        score = fbeta_score(y_true[:,column],y_pred[:,column],beta,average='weighted')
        score_list.append(score)
    f1score_numpy = np.asarray(score_list)
    f1score_numpy = f1score_numpy[f1score_numpy<1]
    f1score = gmean(f1score_numpy)
    return  f1score

In [12]:
# F1 Scores
multi_f1 = multioutput_fscore(y_test,y_pred, beta = 1)
overall_accuracy = (y_pred == y_test).mean().mean()

print('Average overall accuracy {0:.2f}% \n'.format(overall_accuracy*100))
print('F1 score (custom definition) {0:.2f}%\n'.format(multi_f1*100))

Average overall accuracy 94.64% 

F1 score (custom definition) 92.93%



C:\Users\Essam\Anaconda2\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [13]:
accuracy = (y_pred == y_test).mean()

print("Accuracy:\n",accuracy)

('Accuracy:\n', request                   0.878719
offer                     0.995042
aid_related               0.733028
medical_help              0.919336
medical_products          0.952136
search_and_rescue         0.975591
security                  0.982838
military                  0.971014
child_alone               1.000000
water                     0.944317
food                      0.918383
shelter                   0.924867
clothing                  0.986651
money                     0.980931
missing_people            0.988558
refugees                  0.967963
death                     0.958429
other_aid                 0.868230
infrastructure_related    0.937071
transport                 0.955378
buildings                 0.951182
electricity               0.977689
tools                     0.993898
hospitals                 0.990847
shops                     0.995805
aid_centers               0.987223
other_infrastructure      0.956903
weather_related           0.840961
floo

In [14]:
results_dic1={}
for column in y_pred.columns:
    print(column)
    print(classification_report(y_test[column], y_pred[column]))
    print('_____________________________________________________')

    results_dic1[column]=classification_report(y_test[column], y_pred[column])

request
              precision    recall  f1-score   support

           0       0.88      0.98      0.93      4349
           1       0.82      0.37      0.51       895

   micro avg       0.88      0.88      0.88      5244
   macro avg       0.85      0.68      0.72      5244
weighted avg       0.87      0.88      0.86      5244

_____________________________________________________
offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5218
           1       0.00      0.00      0.00        26

   micro avg       1.00      1.00      1.00      5244
   macro avg       0.50      0.50      0.50      5244
weighted avg       0.99      1.00      0.99      5244

_____________________________________________________
aid_related
              precision    recall  f1-score   support

           0       0.73      0.88      0.80      3113
           1       0.75      0.51      0.61      2131

   micro avg       0.73      0.73      0.73     

C:\Users\Essam\Anaconda2\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


search_and_rescue
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      5117
           1       0.43      0.02      0.04       127

   micro avg       0.98      0.98      0.98      5244
   macro avg       0.70      0.51      0.52      5244
weighted avg       0.96      0.98      0.96      5244

_____________________________________________________
security
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      5156
           1       0.25      0.01      0.02        88

   micro avg       0.98      0.98      0.98      5244
   macro avg       0.62      0.51      0.51      5244
weighted avg       0.97      0.98      0.98      5244

_____________________________________________________
military
              precision    recall  f1-score   support

           0       0.97      1.00      0.99      5089
           1       0.59      0.06      0.12       155

   micro avg       0.97      0.97     

In [15]:
# Get names of all categories
category_names = y_test.columns.tolist()

y_pred_df = pd.DataFrame(y_pred, columns = category_names)
y_pred_df.head()

,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,water,...,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report,direct,news
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


### 6. Improve your model
Use grid search to find better parameters. 

#### Train using Grid Cross Validation

In [16]:
start_time  = datetime.datetime.now()
pipeline = Pipeline([
    ('vectorizer', CountVectorizer(tokenizer=tokenize)), #create the CountVectorizer object
    ('tfidf', TfidfTransformer()), #create Tfidftransformer object    
    ('clf', MultiOutputClassifier(RandomForestClassifier())) #create the Classifier object
]) 

In [17]:
pipeline.get_params().keys()

['clf__estimator__class_weight',
 'vectorizer__ngram_range',
 'vectorizer__max_features',
 'clf__estimator__max_leaf_nodes',
 'clf__estimator__criterion',
 'vectorizer__max_df',
 'tfidf__use_idf',
 'clf__estimator__n_jobs',
 'clf__n_jobs',
 'vectorizer__encoding',
 'clf__estimator__random_state',
 'clf__estimator__min_impurity_split',
 'tfidf__smooth_idf',
 'tfidf__sublinear_tf',
 'clf',
 'clf__estimator',
 'vectorizer__input',
 'clf__estimator__min_weight_fraction_leaf',
 'memory',
 'clf__estimator__oob_score',
 'vectorizer__preprocessor',
 'vectorizer',
 'clf__estimator__bootstrap',
 'vectorizer__min_df',
 'clf__estimator__min_samples_split',
 'vectorizer__token_pattern',
 'vectorizer__analyzer',
 'vectorizer__binary',
 'vectorizer__lowercase',
 'clf__estimator__min_samples_leaf',
 'vectorizer__tokenizer',
 'clf__estimator__max_depth',
 'tfidf__norm',
 'vectorizer__stop_words',
 'vectorizer__vocabulary',
 'vectorizer__dtype',
 'clf__estimator__verbose',
 'clf__estimator__warm_start',

In [18]:
parameters = {    
        'tfidf__use_idf': (True, False), 
        'clf__estimator__n_estimators': [50, 100],
        'clf__estimator__min_samples_split': [2,4] }

In [19]:
grid_cv = GridSearchCV(pipeline, param_grid=parameters, cv=5,verbose=3,n_jobs=-1)

In [20]:
grid_cv.fit(X_train, y_train)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
C:\Users\Essam\Anaconda2\lib\site-packages\sklearn\externals\joblib\externals\loky\process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed: 27.1min
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed: 61.3min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('vectorizer', CountVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
    ..._score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=None))]),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'tfidf__use_idf': (True, False), 'clf__estimator__n_estimators': [50, 100], 'clf__estimator__min_samples_split': [2, 4]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=3)

In [21]:
end_time = datetime.datetime.now()
print ('Training of Pipeline Done..., Time: %d' % ((end_time - start_time).seconds/60),'minutes')

('Training of Pipeline Done..., Time: 70', 'minutes')


#### Identify Best Cross Validation parameters 

In [22]:
grid_cv.best_score_
best_parameters=grid_cv.best_estimator_.get_params()

In [23]:
best_parameters

{'clf': MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
             max_depth=None, max_features='auto', max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
             oob_score=False, random_state=None, verbose=0,
             warm_start=False),
            n_jobs=None),
 'clf__estimator': RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
             max_depth=None, max_features='auto', max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
             oob_score=False, random_state=None, verbose=0,
             warm_start=False),
 'clf__estimator__bootstrap': True,
 'clf__estimato

In [24]:
#https://www.programcreek.com/python/example/91151/sklearn.model_selection.GridSearchCV

for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

	clf__estimator__min_samples_split: 2
	clf__estimator__n_estimators: 100
	tfidf__use_idf: False


#### Train model using Best Cross Validation parameters

In [25]:
pipeline = Pipeline([
    ('vectorizer', CountVectorizer(tokenizer=tokenize)), #create the CountVectorizer object
    ('tfidf', TfidfTransformer(use_idf=True)), #create Tfidftransformer object    
    ('clf', MultiOutputClassifier(RandomForestClassifier(min_samples_split= 4,n_estimators= 100),\
        n_jobs=-1)) #create the Classifier object
]) 

In [26]:
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vectorizer', CountVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
    ...ob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=-1))])

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

#### Test model using Best Cross Validation parameters

In [27]:
# test the classifier
y_pred = pipeline.predict(X_test)

In [28]:
#Convert prediction numpy into dataframe
y_test=pd.DataFrame(data=y_test,columns=Y.columns)
y_pred=pd.DataFrame(data=y_pred,columns=Y.columns)

In [29]:
results_dic2={}

for column in y_pred.columns:
    print(column)
    print(classification_report(y_test[column], y_pred[column]))
    print('_____________________________________________________')
    
    results_dic2[column]=classification_report(y_test[column], y_pred[column])


request
              precision    recall  f1-score   support

           0       0.89      0.99      0.94      4349
           1       0.90      0.43      0.58       895

   micro avg       0.89      0.89      0.89      5244
   macro avg       0.90      0.71      0.76      5244
weighted avg       0.89      0.89      0.88      5244

_____________________________________________________
offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5218
           1       0.00      0.00      0.00        26

   micro avg       1.00      1.00      1.00      5244
   macro avg       0.50      0.50      0.50      5244
weighted avg       0.99      1.00      0.99      5244

_____________________________________________________
aid_related
              precision    recall  f1-score   support

           0       0.77      0.88      0.82      3113
           1       0.78      0.62      0.69      2131

   micro avg       0.77      0.77      0.77     

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

#### Train using Grid Cross Validation

In [30]:
start_time  = datetime.datetime.now()

In [31]:
pipeline = Pipeline([
    ('vectorizer', CountVectorizer(tokenizer=tokenize)), #create the CountVectorizer object
    ('tfidf', TfidfTransformer()), #create Tfidftransformer object    
    ('clf', MultiOutputClassifier(OneVsRestClassifier(LinearSVC()))) #create the Classifier object
])


In [32]:
pipeline.get_params()

{'clf': MultiOutputClassifier(estimator=OneVsRestClassifier(estimator=LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
      intercept_scaling=1, loss='squared_hinge', max_iter=1000,
      multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
      verbose=0),
           n_jobs=None),
            n_jobs=None),
 'clf__estimator': OneVsRestClassifier(estimator=LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
      intercept_scaling=1, loss='squared_hinge', max_iter=1000,
      multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
      verbose=0),
           n_jobs=None),
 'clf__estimator__estimator': LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
      intercept_scaling=1, loss='squared_hinge', max_iter=1000,
      multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
      verbose=0),
 'clf__estimator__estimator__C': 1.0,
 'clf__estimator__estimator__class_weight': None,
 'clf__estimator__estimator__d

In [33]:
parameters = {    
        'tfidf__use_idf': (True, False),
        'vectorizer__max_df': [0.8,0.9,1.0],
        'vectorizer__ngram_range': ((1, 1),(1,2)),
        'clf__estimator__estimator__C': [0.01,0.1, 1,10,100]
  }

In [34]:
grid_cv = GridSearchCV(pipeline, param_grid=parameters, cv=5,verbose=3,n_jobs=-1)

In [35]:
grid_cv.fit(X_train, y_train)

Fitting 5 folds for each of 60 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed: 13.0min
[Parallel(n_jobs=-1)]: Done 272 tasks      | elapsed: 53.8min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed: 66.7min finished
C:\Users\Essam\Anaconda2\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 0 is present in all training examples.
  str(classes[c]))


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('vectorizer', CountVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
    ...', random_state=None, tol=0.0001,
     verbose=0),
          n_jobs=None),
           n_jobs=None))]),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'clf__estimator__estimator__C': [0.01, 0.1, 1, 10, 100], 'vectorizer__ngram_range': ((1, 1), (1, 2)), 'tfidf__use_idf': (True, False), 'vectorizer__max_df': [0.8, 0.9, 1.0]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=3)

In [36]:
end_time = datetime.datetime.now()
print ('Training of Pipeline Done..., Time: %d' % ((end_time - start_time).seconds/60),'minutes')

('Training of Pipeline Done..., Time: 67', 'minutes')


####  Identify Best Cross Validation parameters

In [37]:
grid_cv.best_score_

0.4029658592408926

In [38]:
best_parameters=grid_cv.best_estimator_.get_params()

In [39]:
best_parameters

{'clf': MultiOutputClassifier(estimator=OneVsRestClassifier(estimator=LinearSVC(C=1, class_weight=None, dual=True, fit_intercept=True,
      intercept_scaling=1, loss='squared_hinge', max_iter=1000,
      multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
      verbose=0),
           n_jobs=None),
            n_jobs=None),
 'clf__estimator': OneVsRestClassifier(estimator=LinearSVC(C=1, class_weight=None, dual=True, fit_intercept=True,
      intercept_scaling=1, loss='squared_hinge', max_iter=1000,
      multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
      verbose=0),
           n_jobs=None),
 'clf__estimator__estimator': LinearSVC(C=1, class_weight=None, dual=True, fit_intercept=True,
      intercept_scaling=1, loss='squared_hinge', max_iter=1000,
      multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
      verbose=0),
 'clf__estimator__estimator__C': 1,
 'clf__estimator__estimator__class_weight': None,
 'clf__estimator__estimator__dual': Tr

In [40]:
#https://www.programcreek.com/python/example/91151/sklearn.model_selection.GridSearchCV

for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

	clf__estimator__estimator__C: 1
	tfidf__use_idf: False
	vectorizer__max_df: 0.8
	vectorizer__ngram_range: (1, 1)


#### Train model using Best Cross Validation parameters

In [41]:
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vectorizer', CountVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
    ...', random_state=None, tol=0.0001,
     verbose=0),
          n_jobs=None),
           n_jobs=None))])

In [42]:
# test the classifier
y_pred = pipeline.predict(X_test)

In [43]:
#Convert prediction numpy into dataframe
y_test=pd.DataFrame(data=y_test,columns=Y.columns)
y_pred=pd.DataFrame(data=y_pred,columns=Y.columns)

In [44]:
results_dic={}

for column in y_pred.columns:
    print(column)
    print(classification_report(y_test[column], y_pred[column]))
    print('_____________________________________________________')
    
    results_dic[column]=classification_report(y_test[column], y_pred[column],output_dict=True)['weighted avg']


request
              precision    recall  f1-score   support

           0       0.92      0.97      0.94      4349
           1       0.78      0.60      0.68       895

   micro avg       0.90      0.90      0.90      5244
   macro avg       0.85      0.78      0.81      5244
weighted avg       0.90      0.90      0.90      5244

_____________________________________________________
offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5218
           1       0.00      0.00      0.00        26

   micro avg       1.00      1.00      1.00      5244
   macro avg       0.50      0.50      0.50      5244
weighted avg       0.99      1.00      0.99      5244

_____________________________________________________
aid_related
              precision    recall  f1-score   support

           0       0.80      0.82      0.81      3113
           1       0.72      0.70      0.71      2131

   micro avg       0.77      0.77      0.77     

In [45]:
results_dic

{'aid_centers': {'f1-score': 0.9815936783760634,
  'precision': 0.981365653804275,
  'recall': 0.9872234935163997,
  'support': 5244L},
 'aid_related': {'f1-score': 0.7688370046921121,
  'precision': 0.7684795518969728,
  'recall': 0.7694508009153318,
  'support': 5244L},
 'buildings': {'f1-score': 0.9535387159848336,
  'precision': 0.9537940639048424,
  'recall': 0.9601449275362319,
  'support': 5244L},
 'child_alone': {'f1-score': 1.0,
  'precision': 1.0,
  'recall': 1.0,
  'support': 5244L},
 'clothing': {'f1-score': 0.9868742058766515,
  'precision': 0.9869311663832321,
  'recall': 0.9889397406559878,
  'support': 5244L},
 'cold': {'f1-score': 0.9789206080381652,
  'precision': 0.9794801867862285,
  'recall': 0.9824561403508771,
  'support': 5244L},
 'death': {'f1-score': 0.9672827189247415,
  'precision': 0.9678659610392815,
  'recall': 0.9710144927536232,
  'support': 5244L},
 'direct': {'f1-score': 0.9690553006348562,
  'precision': 0.9691710272974121,
  'recall': 0.969107551487

### 9. Export your model as a pickle file

In [47]:
pickle_out = open('classifier.pkl','wb')
pickle.dump(pipeline, pickle_out)



### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.